![Henry Logo](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Ejercicio Sencillo Agent2Agent con Batman + RAG

## Objetivo
Implementar un flujo **agent2agent** minimalista y util en AI Engineering aplicado:
- **Agent 1 (Detective Retriever)**: recupera evidencia desde la base vectorial.
- **Agent 2 (Profesor Synthesizer)**: construye la respuesta final sustentada en contexto.

Si la respuesta no esta suficientemente fundamentada, el orquestador solicita un segundo ciclo de retrieval.

## Diagrama del ejercicio

```mermaid
flowchart TD
  A["Pregunta del usuario"] --> B["Detective Retriever Agent"]
  B --> C["Contexto recuperado"]
  C --> D["Profesor Synthesizer Agent"]
  D --> E["Evaluacion de grounding"]
  E -->|"bajo"| B
  E -->|"suficiente"| F["Respuesta final"]
```

## Marco teorico: El patron Agent2Agent

### Definicion formal

El patron **Agent2Agent** (A2A) organiza un sistema en torno a dos o mas agentes con responsabilidades claramente separadas que colaboran a traves de un orquestador. En este ejercicio:

- **Agent 1 — Detective Retriever**: responsable exclusivamente de la recuperacion de evidencia desde la base vectorial. No genera texto natural; su output es un conjunto de documentos con metadata.
- **Agent 2 — Profesor Synthesizer**: responsable de la generacion de respuesta. Recibe contexto (los documentos del Agent 1) y produce una respuesta citada y coherente. No interactua con la base vectorial.
- **Orquestador**: media la comunicacion entre agentes y aplica un control de calidad (grounding check) antes de entregar la respuesta final. Si la calidad es insuficiente, solicita un segundo ciclo.

### Ventajas sobre un pipeline monolitico

| Aspecto | Monolitico (todo en un paso) | Agent2Agent |
|---|---|---|
| **Trazabilidad** | Dificil saber que parte del pipeline fallo | Se puede auditar el output de cada agente independientemente |
| **Modularidad** | Cambiar el retriever implica cambiar todo | Se puede reemplazar un agente sin afectar al otro |
| **Control de calidad** | El grounding check se aplica al final, sin opcion de remediar | El orquestador puede solicitar un segundo ciclo con parametros ajustados |
| **Depuracion** | Los logs mezclan retrieval y generacion | El dialogo entre agentes genera una traza natural y legible |

### Relacion con "tool use" en agentes LLM

Este patron es conceptualmente analogo al **tool use** de agentes LLM (como el paradigma ReAct), pero con una diferencia clave:
- En tool use, un agente LLM invoca herramientas genericas (busqueda, calculadora, API).
- En Agent2Agent, cada "herramienta" es un agente especializado con su propio system prompt, logica interna y criterios de calidad.

La ventaja del Agent2Agent es que cada agente puede tener un prompt optimizado para su tarea especifica, mientras que en tool use el agente principal debe manejar toda la complejidad en un solo prompt.

### El grounding check como gate de calidad

El segundo ciclo (cuando `groundedness < 0.18`) no es simplemente "reintentar": el orquestador modifica la query para enfocarse en evidencia concreta y cambia el modo del synthesizer a "estricto". Esto es un patron de **retroalimentacion correctiva**, donde el sistema intenta mejorar su output antes de entregarlo al usuario. En produccion, el numero de reintentos deberia estar acotado (tipicamente 1-2) para evitar loops infinitos y latencia excesiva.

In [1]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import sys
import time
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.common import generate_answer
from scripts.evaluation import groundedness_score
from scripts.vector_store_lab import build_index_from_json

OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
DATA_PATH = ROOT / 'data' / 'batman_comics.json'

/Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db, chunks, index_stats, chunk_stats = build_index_from_json(
    json_path=DATA_PATH,
    persist_dir=OUTPUTS_DIR / 'chroma_agent2agent_batman',
    collection_name='agent2agent_batman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Index stats:', index_stats)
print('Chunk stats:', chunk_stats)

Index stats: {'collection': 'agent2agent_batman', 'indexed_chunks': 38, 'embedding_provider': 'local:hash-embedding'}
Chunk stats: {'chunk_count': 38, 'unique_sources': 12, 'avg_chars_per_chunk': 682.5, 'max_chars_per_chunk': 792, 'themes_distribution': {'origen': 3, 'villanos': 3, 'legado': 3, 'conspiracion': 3, 'derrota': 3, 'misterio': 3, 'relaciones': 3, 'equipo': 3, 'escenario': 3, 'mentoria': 4, 'recursos': 4, 'filosofia': 3}, 'hero_distribution': {'batman': 38}}


In [3]:
@dataclass
class DetectiveRetrieverAgent:
    vector_db: object
    embedding_model: str = 'text-embedding-3-small'

    def retrieve(self, query: str, k: int = 5) -> tuple[list[dict], str]:
        docs, provider = self.vector_db.query(
            query_text=query,
            n_results=k,
            embedding_model=self.embedding_model,
        )
        return docs, provider


@dataclass
class ProfesorSynthesizerAgent:
    model: str = 'gpt-5-mini'

    def answer(self, question: str, docs: list[dict], mode: str = 'normal') -> tuple[str, str]:
        contexts = [str(doc.get('text', '')) for doc in docs]
        if mode == 'estricto':
            system_prompt = (
                'Modo estricto de verificabilidad: responde solo con evidencia del contexto, '
                'no inventes datos, y cita [D#].'
            )
        else:
            system_prompt = (
                'Eres profesor de AI Engineering aplicado. Explica con claridad y rigor tecnico, '
                'limitandote al contexto recuperado y citando [D#].'
            )

        return generate_answer(
            query=question,
            contexts=contexts,
            model=self.model,
            system_prompt=system_prompt,
        )


class Agent2AgentOrchestrator:
    def __init__(self, retriever: DetectiveRetrieverAgent, synthesizer: ProfesorSynthesizerAgent) -> None:
        self.retriever = retriever
        self.synthesizer = synthesizer

    def run(self, question: str, k: int = 5, grounding_threshold: float = 0.18) -> dict:
        t0 = time.perf_counter()
        dialogue = []

        docs, retrieval_provider = self.retriever.retrieve(question, k=k)
        dialogue.append({
            'agent': 'DetectiveRetrieverAgent',
            'message': f'Recuperados {len(docs)} documentos con provider={retrieval_provider}',
        })

        answer, llm_provider = self.synthesizer.answer(question=question, docs=docs, mode='normal')
        score = groundedness_score(answer=answer, contexts=[doc.get('text', '') for doc in docs])
        dialogue.append({
            'agent': 'ProfesorSynthesizerAgent',
            'message': f'Borrador generado con provider={llm_provider} y groundedness={score}',
        })

        if score < grounding_threshold:
            refined_query = question + ' Enfocate en evidencia canonica concreta y eventos verificables.'
            docs_refined, retrieval_provider_2 = self.retriever.retrieve(refined_query, k=k + 2)
            dialogue.append({
                'agent': 'DetectiveRetrieverAgent',
                'message': (
                    f'Retrieval de refuerzo: {len(docs_refined)} docs con provider={retrieval_provider_2}'
                ),
            })
            docs = docs_refined
            answer, llm_provider = self.synthesizer.answer(question=question, docs=docs, mode='estricto')
            score = groundedness_score(answer=answer, contexts=[doc.get('text', '') for doc in docs])
            dialogue.append({
                'agent': 'ProfesorSynthesizerAgent',
                'message': f'Respuesta final regenerada con groundedness={score}',
            })

        latency = round(time.perf_counter() - t0, 4)
        return {
            'question': question,
            'answer': answer,
            'groundedness': score,
            'retrieved_docs': len(docs),
            'latency_seconds': latency,
            'llm_provider': llm_provider,
            'retrieval_provider': retrieval_provider,
            'dialogue': dialogue,
        }

In [4]:
retriever_agent = DetectiveRetrieverAgent(vector_db=db, embedding_model='text-embedding-3-small')
synthesizer_agent = ProfesorSynthesizerAgent(model='gpt-5-mini')
orchestrator = Agent2AgentOrchestrator(retriever=retriever_agent, synthesizer=synthesizer_agent)

print('Agent2Agent pipeline listo.')

Agent2Agent pipeline listo.


In [5]:
query = 'Explica como Batman combina investigacion y estrategia para enfrentar amenazas como Bane y el Joker.'
result = orchestrator.run(question=query, k=5, grounding_threshold=0.18)

pd.DataFrame(result['dialogue'])

,agent,message
0,DetectiveRetrieverAgent,Recuperados 5 documentos con provider=local:ha...
1,ProfesorSynthesizerAgent,Borrador generado con provider=local:fallback-...


In [6]:
print('Pregunta:')
print(result['question'])
print('\nRespuesta:')
print(result['answer'])
print('\nMetricas:')
print({
    'groundedness': result['groundedness'],
    'retrieved_docs': result['retrieved_docs'],
    'latency_seconds': result['latency_seconds'],
    'llm_provider': result['llm_provider'],
    'retrieval_provider': result['retrieval_provider'],
})

Pregunta:
Explica como Batman combina investigacion y estrategia para enfrentar amenazas como Bane y el Joker.

Respuesta:
Respuesta local fallback (sin llamada a OpenAI):
- The Killing Joke de Alan Moore es considerada la historia definitiva sobre la relacion entre Batman y el Joker [D4]
- La premisa central del Joker es simple y aterradora: cualquier persona esta a un mal dia de volverse como el [D4]
- Para probarlo, el Joker dispara a Barbara Gordon (Batgirl) en la columna vertebral, la fotografa mientras sufre, y secuestra al comisionado Gordon para someterlo a una noche de tortura psicologica en un parque de diversiones abandonado [D4]

Metricas:
{'groundedness': 0.9032, 'retrieved_docs': 5, 'latency_seconds': 0.0019, 'llm_provider': 'local:fallback-summary', 'retrieval_provider': 'local:hash-embedding'}


## Mini-practica guiada

Ejecuta estas consultas y compara el comportamiento del dialogo entre agentes:
1. `Que leccion deja Knightfall sobre delegacion y resiliencia?`
2. `Como se relaciona la filosofia de no matar con decisiones tacticas en Gotham?`
3. `Por que Batman es efectivo en la Liga de la Justicia sin superpoderes?`

In [7]:
exercise_queries = [
    'Que leccion deja Knightfall sobre delegacion y resiliencia?',
    'Como se relaciona la filosofia de no matar con decisiones tacticas en Gotham?',
    'Por que Batman es efectivo en la Liga de la Justicia sin superpoderes?',
]

rows = []
for q in exercise_queries:
    run = orchestrator.run(question=q, k=5, grounding_threshold=0.18)
    rows.append({
        'question': q,
        'groundedness': run['groundedness'],
        'retrieved_docs': run['retrieved_docs'],
        'latency_seconds': run['latency_seconds'],
        'llm_provider': run['llm_provider'],
        'retrieval_provider': run['retrieval_provider'],
    })

exercise_df = pd.DataFrame(rows)
exercise_df

,question,groundedness,retrieved_docs,latency_seconds,llm_provider,retrieval_provider
0,Que leccion deja Knightfall sobre delegacion y...,0.9118,5,0.0007,local:fallback-summary,local:hash-embedding
1,Como se relaciona la filosofia de no matar con...,0.8987,5,0.0006,local:fallback-summary,local:hash-embedding
2,Por que Batman es efectivo en la Liga de la Ju...,0.8929,5,0.0006,local:fallback-summary,local:hash-embedding


In [8]:
csv_path = OUTPUTS_DIR / 'agent2agent_exercise_results.csv'
exercise_df.to_csv(csv_path, index=False)
print(f'Saved: {csv_path}')

Saved: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/agent2agent_exercise_results.csv


## Cierre didactico

### Conceptos clave de esta notebook

- **Separacion de responsabilidades mejora auditabilidad**: cuando retrieval y generacion son agentes separados, puedes evaluar cada uno independientemente. Si la respuesta es mala, sabes si el problema fue el retrieval (documentos irrelevantes) o la generacion (mala sintesis del contexto).
- **El dialogo entre agentes es una traza natural**: la lista de `dialogue` no es solo decorativa — en produccion, ese log es tu herramienta principal de debugging. Incluyelo siempre.
- **El segundo ciclo correctivo tiene costo**: cada ciclo adicional multiplica la latencia y el costo de API. El threshold de grounding (0.18) y el numero maximo de ciclos deben calibrarse para el trade-off calidad/costo de tu caso de uso.
- **Agent2Agent simple > multi-agente complejo** (en la mayoria de casos): antes de disenar un sistema con 5+ agentes, verifica que un flujo de 2 agentes con un orquestador no resuelve tu problema igualmente bien.

### Conexion con la siguiente notebook

En la siguiente notebook extendemos este patron con **especializacion por roles**: multiples agentes especialistas (timeline, villains, strategy) son coordinados por un router heuristico. Esto introduce la pregunta: *cuando vale la pena especializar agentes vs tener uno generalista?*